In [2]:
import numpy as np 
import pandas as pd
import scistreecna as scna


In [3]:
# load .csv file 
reads, cell_names, site_names = scna.util.read_csv('../examples/test_data_reads.csv')

In [5]:
# run inference
scistreecna_tree, scistreecna_geno = scna.infer(reads,
                                              cell_names=cell_names,  # cell names
                                              ado=0.1,  # allelic dropout rate
                                              seq_error=0.01,   # sequencing error
                                              cn_noise=0.05,    # copy number noise
                                              cn_min=1, # minimum copy number (>=1) 
                                              cn_max=5, # maximum copy number
                                              tree_batch_size=128,  # number of trees evaluated together
                                              node_batch_size=256,  # number of nodes evaluated together 
                                              verbose=True)  # print logs

─────────────────────────────────────────────────── ScisTreeCNA ───────────────────────────────────────────────────

#Cell: 60 #Site: 100

CN_MIN: 1 CN_MAX: 5 ADO: 0.1 SEQ_ERR: 0.01 CN_NOISE: 0.05

TREE_BATCH_SIZE: 128 NODE_BATCH_SIZE: 256

────────────────────────────────────────────────── Local Search ───────────────────────────────────────────────────

...

[23:52:47] [Iteration 31]  Likelihood: -12405.9390                                               ]8;id=364774;file:///home/haz19024/miniconda3/envs/scistreecna/lib/python3.12/site-packages/scistreecna/scistreecna.py\scistreecna.py]8;;\:]8;id=77394;file:///home/haz19024/miniconda3/envs/scistreecna/lib/python3.12/site-packages/scistreecna/scistreecna.py#699\699]8;;\

[23:52:48] [Iteration 32]  Likelihood: -12405.9105                                               ]8;id=269670;file:///home/haz19024/miniconda3/envs/scistreecna/lib/python3.12/site-packages/scistreecna/scistreecna.py\scistreecna.py]8;;\:]8;id=784959;file:///home/haz19024/miniconda3/envs/scistreecna/lib/python3.12/site-packages/scistreecna/scistreecna.py#699\699]8;;\

[23:52:49] [Iteration 33]  Likelihood: -12405.8835                                               ]8;id=800041;file:///home/haz19024/miniconda3/envs/scistreecna/lib/python3.12/site-packages/scistreecna/scistreecna.py\scistreecna.py]8;;\:]8;id=263337;file:///home/haz19024/miniconda3/envs/scistreecna/lib/python3.12/site-packages/scistreecna/scistreecna.py#699\699]8;;\

[23:52:51] Local search complete. Best Likelihood: -12405.883480072021                           ]8;id=989523;file:///home/haz19024/miniconda3/envs/scistreecna/lib/python3.12/site-packages/scistreecna/scistreecna.py\scistreecna.py]8;;\:]8;id=84916;file:///home/haz19024/miniconda3/envs/scistreecna/lib/python3.12/site-packages/scistreecna/scistreecna.py#686\686]8;;\

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────